# Establish Secure Connection to Snowflake

### Create a conda virtual environment and install snowpark and other dependencies
`conda create --name snowpark_env -c https://repo.anaconda.com/pkgs/snowflake python=3.8 numpy pandas cachetools spacy scikit-learn`
<br>
`conda activate snowpark_env`
<br>
`pip install "snowflake-snowpark-python[pandas]"`
<br>
`python -m spacy download en_core_web_sm`

### packages that Snowflake Anaconda doesn't support yet
`pip install spacytextblob`
<br>
`pip install pytextrank`
### Add virtual env to ipykernel for Jupyter Notebook to use
`conda install -c anaconda ipykernel`
<br>
`python -m ipykernel install --user --name=snowpark_env`
### Create stage "python_load" and stage "model_data" to store Python UDFs and trained model files
Instructions on how to create a stage on Snowflake: https://hevodata.com/learn/snowflake-stages/
### Upload spacy pretrained NLP pipeline to a named stage from local computer
`put file://C:\Users\an.jiang\src\snowpark_demo\spacy_trained_models\model_for_upload\en_core_web_sm.zip @model_data AUTO_COMPRESS=TRUE OVERWRITE = TRUE;`
<br>
How to use PUSH command in snowsql cli: https://docs.snowflake.com/en/sql-reference/sql/put.html
### Use Snowflake Dashboard and Chart tools of Snowsight to visualize Zendesk ticket sentiment trend
https://hevodata.com/learn/snowflake-dashboards/

In [1]:
# Snowpark for Python
import snowflake.snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, col, call_udf
from snowflake.snowpark.types import *

# Others
import sys, string, io, os, math
import zipfile
import pickle
import numpy as np
import pandas as pd
import json
from cachetools import cached

# increase the max number of columns to display - default 20, switch to truncate view if exceeded
pd.set_option('display.max_columns', 50)
# increase the width of the column so we can see more raw text - default 50 characters
pd.set_option('display.max_colwidth', 500)

In [2]:
connection_parameters = {
    "account": 'wne',
    "user": 'an.jiang@imaginelearning.com',
    "authenticator": 'externalbrowser',
    # "role": os.environ["SNOWFLAKE_ROLE"]
    "warehouse": 'SQL_WH',
    "database": 'SANDBOX',
    "schema": 'SCRATCH'
  }
# connection_parameters = json.load(open(r'C:\Users\an.jiang\src\snowflake_credentials.json'))

In [3]:
# Create Snowflake Session object
session = Session.builder.configs(connection_parameters).create()
print(session.sql("select current_warehouse(), current_database(), current_schema()").collect())

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
[Row(CURRENT_WAREHOUSE()='SQL_WH', CURRENT_DATABASE()='SANDBOX', CURRENT_SCHEMA()='SCRATCH')]


***

===================================================================================================================

# Read unstructured Zendesk data 

In [4]:
df_zendesk = session.table('zendesk_tickets')
df_zendesk.limit(5).toPandas()

,ID,URL,EXTERNAL_ID,TYPE,SUBJECT,RAW_SUBJECT,DESCRIPTION,PRIORITY,STATUS,RECIPIENT,REQUESTER_ID,SUBMITTER_ID,ASSIGNEE_ID,ORGANIZATION_ID,GROUP_ID,COLLABORATOR_IDS,FOLLOWER_IDS,EMAIL_CC_IDS,FORUM_TOPIC_ID,PROBLEM_ID,HAS_INCIDENTS,DUE_AT,TAGS,VIA,CUSTOM_FIELDS,SATISFACTION_RATING,SHARING_AGREEMENT_IDS,FOLLOWUP_IDS,TICKET_FORM_ID,BRAND_ID,ALLOW_CHANNELBACK,ALLOW_ATTACHMENTS,IS_PUBLIC,CREATED_AT,UPDATED_AT,ROW_CHECKSUM
0,2010904,https://edgenuity.zendesk.com/api/v2/tickets/2010904.json,None,question,Course Request,Course Request,"<div align=""center""><div align=""center"">\n\n<table border=""1"" cellspacing=""0"" cellpadding=""0"" style=""max-width: 6.25in; background-image: initial; background-size: initial; background-repeat: initial; background-attachment: initial; background-origin: initial; background-clip: initial; background-color: rgba(255, 255, 255, 1); border-collapse: collapse; border: none"">\n <tbody><tr>\n <td style=""border-top: 1pt solid rgba(221, 221, 221, 1); border-right: 1pt solid rgba(221, 221, 221, 1); bor...",normal,closed,admissions@edgenuity.com,362143489268,362143489268,1507094000222,1.500442e+12,360004538234,[],[],[],NaN,NaN,False,NaT,"[\n ""admissions_course_request"",\n ""asa_individual_public_course_request"",\n ""assigned_to_admissions"",\n ""dg-macro-1500031315001"",\n ""did_31935"",\n ""is_admissions_survey_sent"",\n ""is_help_center"",\n ""product_courseware"",\n ""realm_05"",\n ""standard_school""\n]","{\n ""channel"": ""email"",\n ""source"": {\n ""from"": {\n ""address"": ""noreply@edgenuity.com"",\n ""name"": ""Reply No""\n },\n ""to"": {\n ""address"": ""admissions@edgenuity.com"",\n ""name"": ""Imagine Learning""\n }\n }\n}","[\n {\n ""id"": 360053997313\n },\n {\n ""id"": 360053495554\n },\n {\n ""id"": 360046824194\n },\n {\n ""id"": 360052996354\n },\n {\n ""id"": 360031704834\n },\n {\n ""id"": 360053998853\n },\n {\n ""id"": 360033966853\n },\n {\n ""id"": 1900006981125\n },\n {\n ""id"": 1500003320582\n },\n {\n ""id"": 1500007156742,\n ""value"": ""false""\n },\n {\n ""id"": 360000070407\n },\n {\n ""id"": 360047024393\n },\n {\n ""id"": 1900000749065\n },\n {\n ""i...","{\n ""score"": ""unoffered""\n}",[],[],1500000574562,360002067674,False,True,True,2022-03-28 23:41:34,2022-04-02 13:01:38,25913f74e72be9f2c3e91f3cedb5ed9a8cdf40b4e69019dae3e14baa135e40b5
1,2010996,https://edgenuity.zendesk.com/api/v2/tickets/2010996.json,None,None,Sync CompleteSuccess - - 4821160 - MathFacts,Sync CompleteSuccess - - 4821160 - MathFacts,"# Sync CompleteFailure\n\n***Session ID: 2022-03-29T05:02:00.587Z***\n***Job ID: 426f37a2-1318-45d2-9a31-95e4422496b0***\n\nFatal error processing 4821160 session 2022-03-29T05:02:00.587Z for MathFacts. \nImportType: oneroster11\n\nUploaded Files:\n\timportFiles/4821160/MathFacts/2022-03-29T05:02:00.587Z/oneroster1_1.zip\n\n\n\nFiles copied to staging:\n\timportFiles/4821160/MathFacts/2022-03-29T05:02:00.587Z/OneRoster_errors.csv\nFatal Error: TypeError\n{""errorType"":""TypeError"",""errorMessag...",normal,closed,None,1500104837902,1500104837902,426441101414,3.705828e+11,360015595214,"[\n ""1500195863142"",\n ""360464355887""\n]","[\n ""1500195863142""\n]","[\n ""360464355887""\n]",NaN,NaN,False,NaT,"[\n ""assign_il_eq_2"",\n ""clever_pause"",\n ""imagine_learning_internal_communication"",\n ""products_math_facts"",\n ""rostering"",\n ""rostering_frequency_recurring"",\n ""rostering_sync"",\n ""rostering_sync_failed"",\n ""rosteringlevel_assisted"",\n ""sso_classlink"",\n ""sync_complete_failure"",\n ""sync_complete_success"",\n ""tasks_present"",\n ""tasks_remain"",\n ""ticket_is_created_private""\n]","{\n ""channel"": ""api"",\n ""source"": {\n ""from"": null,\n ""to"": null\n }\n}","[\n {\n ""id"": 360053997313\n },\n {\n ""id"": 360053495554,\n ""value"": ""Sync CompleteSuccess - - 4821160 - MathFacts""\n },\n {\n ""id"": 360046824194\n },\n {\n ""id"": 360052996354\n },\n {\n ""id"": 360031704834\n },\n {\n ""id"": 360053998853\n },\n {\n ""id"": 360033966853\n 

In [5]:
df_zendesk = df_zendesk.toPandas()

In [6]:
# To avoid reading the table multiple times while testing, download the table to local machine and then read the local file

# df_zendesk.toPandas().to_csv('zendesk_tickets.csv.gz', header=True, index=False, compression='gzip')
# Reading a local file
# df_zendesk = pd.read_csv(r"C:\Users\an.jiang\src\snowpark_demo\zendesk_tickets\zendesk_tickets.csv", low_memory=False)

# Data transformation to extract reviews/comments from raw data
This part should be able to be written as Python UDFs for Snowpark DataFrame
<br>
Here I simply write the transformation process explicitly for simplicity

## Extract comments, reasons and scores in the SATISFACTION_RATING column

In [7]:
rating_dicts = df_zendesk["SATISFACTION_RATING"].apply(json.loads) # Apply json.loads function along each row of the DataFrame(here is PandasSeries)
comments, reasons, scores = rating_dicts.apply(lambda x: x.get('comment', np.nan)), rating_dicts.apply(lambda x: x.get('reason', np.nan)), rating_dicts.apply(lambda x: x.get('score', np.nan))

##  Insert them into the DataFrame

In [8]:
insert_index = df_zendesk.columns.get_loc("SATISFACTION_RATING") + 1
df_zendesk.insert(insert_index, column="SCORE", value = scores)
df_zendesk.insert(insert_index, column="REASON", value = reasons)
df_zendesk.insert(insert_index, column="COMMENT", value = comments)

In [9]:
df_zendesk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2202596 entries, 0 to 2202595
Data columns (total 39 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     int32         
 1   URL                    object        
 2   EXTERNAL_ID            object        
 3   TYPE                   object        
 4   SUBJECT                object        
 5   RAW_SUBJECT            object        
 6   DESCRIPTION            object        
 7   PRIORITY               object        
 8   STATUS                 object        
 9   RECIPIENT              object        
 10  REQUESTER_ID           int64         
 11  SUBMITTER_ID           int64         
 12  ASSIGNEE_ID            float64       
 13  ORGANIZATION_ID        float64       
 14  GROUP_ID               float64       
 15  COLLABORATOR_IDS       object        
 16  FOLLOWER_IDS           object        
 17  EMAIL_CC_IDS           object        
 18  FORUM_TOPIC_ID        

## Filter rows by comment and score columns

### By comment - drop rows with no comments

In [10]:
# replace an empty string and whitespaces ((spaces, tabs and new lines)) with nan
df_zendesk["COMMENT"].replace(r'^\s*$', np.nan, regex=True, inplace=True)
# Drop rows with no comments
df_zendesk.dropna(subset=['COMMENT'], inplace=True)

In [11]:
df_zendesk.shape

(55185, 39)

### By Score - turns out that all comments have been labeled 
"unoffered" and "offered" labels have no corresponding comments, which means we don't have unlabeled data

In [12]:
unique_scores = list(df_zendesk['SCORE'].unique())
print("The unique values for column SCORE are: {}".format(unique_scores))
# The unique values for column SCORE before droppng rows with no comments are:
# ['unoffered' 'offered' 'good' 'bad']

The unique values for column SCORE are: ['good', 'bad']


In [13]:
# select "good" and "bad" as two labels for sentiment
# "unoffered" and "offered" do not provide any sentiment information
df_zendesk_annotated = df_zendesk.loc[df_zendesk["SCORE"].isin(['good', 'bad'])]

### Drop duplicates

In [14]:
# Drop duplicate comments except the first occurence.
df_zendesk_annotated.drop_duplicates(subset=['COMMENT'], keep='first', inplace=True)

### Reset index

In [15]:
# Reset index
df_zendesk_annotated.reset_index(drop=True, inplace=True)

## Select columns

In [16]:
# Select columns
columns = ['ID', 'TYPE', 'SUBJECT', 'DESCRIPTION', 'PRIORITY', 'COMMENT', 'REASON', 'SCORE', 'CREATED_AT', 'UPDATED_AT']
df_zendesk_annotated = df_zendesk_annotated[columns]

In [19]:
df_zendesk_annotated.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45883 entries, 0 to 45882
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           45883 non-null  int32         
 1   TYPE         43042 non-null  object        
 2   SUBJECT      45883 non-null  object        
 3   DESCRIPTION  45883 non-null  object        
 4   PRIORITY     45883 non-null  object        
 5   COMMENT      45883 non-null  object        
 6   REASON       45883 non-null  object        
 7   SCORE        45883 non-null  object        
 8   CREATED_AT   45883 non-null  datetime64[ns]
 9   UPDATED_AT   45883 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int32(1), object(7)
memory usage: 3.3+ MB


In [20]:
# Save a local copy for testing purpose
df_zendesk_annotated.to_csv(r"C:\Users\an.jiang\src\snowpark_demo\data\reviews_annotated.csv", index = False)

## Turn Pandas DataFrame back to Snowpark DataFrame and save it to a table
$\color{red}{\text{Somehow datatime64[ns] datatype will convert to int64 when save as a Snowflake table, need to figure out how to specify datatype}}$
<br>
<font color='green'>Somehow datatime64[ns] datatype will convert to int64 when save as a Snowflake table, need to figure out how to specify datatype</font>

In [21]:
# # create_dataframe accepts list or pandas dataframe as value
# df = session.create_dataframe(df_zendesk_annotated) 
# # To save the contents of a DataFrame to a table
# df.write.mode("overwrite").save_as_table("reviews_annotated")

***

======================================================================================================================

# Explore Historical Data
Lets look at the REVIEWS_ANNOTATED table which has the manually annotated sentiment labels for each customer review.

In [33]:
# Create a DataFrame from data in a table
# df = session.table("reviews_annotated") # Snowpark DataFrame
# df.limit(10).toPandas()
# df = df.toPandas()

df = df_zendesk_annotated

In [34]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45883 entries, 0 to 45882
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           45883 non-null  int32         
 1   TYPE         43042 non-null  object        
 2   SUBJECT      45883 non-null  object        
 3   DESCRIPTION  45883 non-null  object        
 4   PRIORITY     45883 non-null  object        
 5   COMMENT      45883 non-null  object        
 6   REASON       45883 non-null  object        
 7   SCORE        45883 non-null  object        
 8   CREATED_AT   45883 non-null  datetime64[ns]
 9   UPDATED_AT   45883 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int32(1), object(7)
memory usage: 3.3+ MB


# Feature Engineering and Data Pre-Processing Tasks

### Transform the text-based sentiment into numerical values

In [35]:
# Create a Python UDF to bin the rating to sentiment
def convert_rating(x: str) -> int:
    if x == 'good': return 1
    elif x == 'bad': return -1

In [36]:
sentiment_rating = df["SCORE"].apply(convert_rating)

In [37]:
insert_index = df.columns.get_loc("SCORE") + 1
df.insert(insert_index, column="SENTIMENT_RATING", value = sentiment_rating)

In [38]:
df[["ID", "COMMENT", "SCORE", "SENTIMENT_RATING"]].head()

,ID,COMMENT,SCORE,SENTIMENT_RATING
0,3280,"The tech support was wonderful. We were able to fix our problem which was actually occurring on our end but without the help of your tech dept, we wouldn't have figured it out.\r\nThanks so much!",good,1
1,7279,"Very Professional. Try to get back to me for day or two, which is cool. Thank you",good,1
2,9458,The response time was good and the tip is helpful for current and future students.,good,1
3,9968,Wendy was very helpful and solved our issue that the teacher/students have been dealing with for the past 14 weeks. Thank you very much!,good,1
4,14361,Thank you. We will continue to monitor the student.,good,1


### Stop Word Removal
We want to remove text that is relevant for readers but not for our machine learning algorithm. In English for example, this includes punctuation and articles such as a & the - which are typically referred to as stop words. To do this we create a Python UDF and use the spaCy library to process the review text.

In [56]:
nlp = spacy.load("en_core_web_sm") # version 3.3.0 same with spaCy
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'go', 'once', 'well', 'say', 'six', 'yet', 'none', 'yourselves', 'cannot', 'still', 'up', 'within', "'ll", 'seemed', 'wherein', 'own', 'this', 'twenty', 'her', 'using', 'herein', 'his', 'he', 'whereupon', 'if', 'eleven', 'than', 'not', 'whom', 'again', 'from', 'twelve', "'m", 'someone', 'me', 'used', 'doing', 'just', 'latterly', 'ten', 'sixty', '‘ve', 'the', 'whenever', 'all', '’d', 'below', 'four', 'whatever', 'seem', 'see', 'however', 'empty', '‘d', 'themselves', 'down', 'further', 'mine', 'among', 'call', 'everywhere', 'five', 'except', 'part', 'much', 'besides', 'itself', 'its', 'thence', 'neither', 'then', 'eight', 'last', 'front', 'another', 'fifteen', 'such', 'only', 'are', 'hereupon', 'beforehand', 'be', 'onto', 'of', 'so', 'elsewhere', 'him', 'whence', 'why', 'ca', 'you', 'wherever', 'but', 'therefore', 'thus', 'in', 'by', 'becoming', 'anyway', 'ourselves', 'yourself', 'whole', 'without', 'perhaps', 'were', 'already', 'amount', 'somehow', "'re", 'sometime', 'therein', 'hers',

In [39]:
import spacy

def remove_stopwords_vect(raw_text: PandasSeries[str]) -> PandasSeries[str]:
    
    nlp = spacy.load("en_core_web_sm") # version 3.3.0 same with spaCy
    stop_words = nlp.Defaults.stop_words
    
    result = []
    
    for s in raw_text:
        doc = nlp(s)
        text = [str(t.lemma_) for t in doc if 
                t not in stop_words
                and not t.is_punct 
                and not t.is_currency
                and not t.is_space
                and t.lemma_ != '-PRON-']
        result.append(' '.join(token.lower() for token in text))
        
    return pandas.Series(result)

In [40]:
processed_text = remove_stopwords_vect(df["COMMENT"])

In [41]:
insert_index = df.columns.get_loc("COMMENT") + 1
df.insert(insert_index, column="PROCESSED_TEXT", value = processed_text)

__remove_stopwords_vect function may produce empty string in the PROCESSED_TEXT column, need to remove the corresponding rows__

In [42]:
# replace an empty string and whitespaces ((spaces, tabs and new lines)) with nan
df['PROCESSED_TEXT'].replace(r'^\s*$', np.nan, regex=True, inplace=True)
# Drop rows with no processed text
df.dropna(subset=['PROCESSED_TEXT'], inplace=True)

In [43]:
# select columns for training - select all for now
reviews_annotated_df = df
# save the processed dataframe to a table (local table for now)
reviews_annotated_df.to_csv(r"C:\Users\an.jiang\src\snowpark_demo\data\reviews_training.csv", index=False)

---

==============================================================================================================

# Train and Deploy a Sentiment Analysis Model
Let's look at how we are able to execute model training inside Snowflake

### Snowpark code for model training

In [6]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import pickle 

def train_sentiment_model() -> float:
    # retrieve data from processed table
    training_file_path = r"C:\Users\an.jiang\src\snowpark_demo\data\reviews_training.csv"
    df_raw = pd.read_csv(training_file_path)
    
    # split data
    train, test = train_test_split(df_raw, test_size=0.2, random_state=42)
    
    # vectorize the review text
    vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
    train_matrix = vectorizer.fit_transform(train['PROCESSED_TEXT'])
    test_matrix = vectorizer.transform(test['PROCESSED_TEXT'])
    
    #split feature and lable
    x_train = train_matrix
    x_test = test_matrix
    y_train = train['SENTIMENT_RATING']
    y_test = test['SENTIMENT_RATING']
    
    # Regression Model
    lr = LogisticRegression(multi_class='multinomial', max_iter=10000)
    lr.fit(x_train, y_train)
    
    predictions = lr.predict(x_test)
    
    with open(r"C:\Users\an.jiang\src\snowpark_demo\test_models\sentiment_lr.pickle","wb") as f:
        pickle.dump(lr, f)
    with open(r"C:\Users\an.jiang\src\snowpark_demo\test_models\vectorizer_lr.pickle","wb") as f:
        pickle.dump(vectorizer, f)
        
#     # Save vectorized embeddings and model files to stage
#     save_file(session, lr, '@MODEL_DATA/sentiment_large.pickle')
#     save_file(session, vectorizer, '@MODEL_DATA/vectorizer_large.pickle')
    
    return accuracy_score(y_test, predictions)

In [7]:
# run training process
train_sentiment_model()

0.9196857267568748

# Using the trained model - to predict comments with unoffered and offered ratings
Because the current data set has no comments with unoffered and offered tags, we apply pseudo prediction on comments in the training and testing data set for now.

In [2]:
# Create a vectorized UDF to predict sentiment
# session.clear_packages()
# session.clear_imports()
# session.add_import('@MODEL_DATA/sentiment_large.pickle')
# session.add_import('@MODEL_DATA/vectorizer_large.pickle')

def predict_sentiment_vector(sentiment_str):
    model_file_path = r"C:\Users\an.jiang\src\snowpark_demo\test_models\sentiment_lr.pickle"
    vector_file_path = r"C:\Users\an.jiang\src\snowpark_demo\test_models\vectorizer_lr.pickle"

    with open(model_file_path, 'rb') as fm:
        model = pickle.load(fm)
        
    with open(vector_file_path, 'rb') as fv:
        vectorizer = pickle.load(fv)
        
    result = []
    
    for s in sentiment_str:
        matrix = vectorizer.transform([s])
        
        df= pd.DataFrame(model.predict_proba(matrix), columns=('NEGATIVE', 'POSITIVE'))
        
        response = df.loc[0].to_json()
        result.append(json.loads(response))

    return pandas.Series(result)

### Let's score new reviews that have been pre-processed to get the sentiment scores

In [3]:
# Load new preprocessed reviews
new_processed_reviews_path = r"C:\Users\an.jiang\src\snowpark_demo\data\reviews_training.csv"
parse_dates = ['CREATED_AT', 'UPDATED_AT']
df_new_reviews = pd.read_csv(new_processed_reviews_path, parse_dates=parse_dates)
# make predictions on new reviews
sentiment = predict_sentiment_vector(df_new_reviews["PROCESSED_TEXT"])
negative = sentiment.apply(lambda x: x["NEGATIVE"])
positive = sentiment.apply(lambda x: x["POSITIVE"])
# Add sentiment to the dataframe
insert_index = df_new_reviews.columns.get_loc("PROCESSED_TEXT") + 1
df_new_reviews.insert(insert_index, column="POSITIVE", value = positive)
df_new_reviews.insert(insert_index, column="NEGATIVE", value = negative)

# Using spacytextblob library to get positive and negative words from comments 
$\color{red}{\text{Snowflake Anaconda channel doesn't support spacytextblob yet, we may submit a request}}$

In [4]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob


def get_pos_neg_words(sentiment_str: PandasSeries[str]) -> PandasSeries[str]:
    # load a spaCy model, depending on language, scale, etc.
    nlp = spacy.load('en_core_web_sm')
    # add TextBlob to the spaCy pipeline
    nlp.add_pipe('spacytextblob')
    
    total_pos = []
    total_neg = []
    for text in sentiment_str:
        positive_words = []
        negative_words = []
        
        doc = nlp(text)

        for x in doc._.blob.sentiment_assessments.assessments:
            if x[1] > 0:
                positive_words.extend(x[0])
            elif x[1] < 0:
                negative_words.extend(x[0])
            else:
                pass

        total_pos.append(', '.join(set(positive_words)))
        total_neg.append(', '.join(set(negative_words)))
        
    return pandas.Series(total_pos), pandas.Series(total_neg)

In [5]:
# get positive and negative words from new reviews
positive_words_vect, negative_words_vect = get_pos_neg_words(df_new_reviews["PROCESSED_TEXT"])
# Add positive and negative words to the dataframe
insert_index = df_new_reviews.columns.get_loc("POSITIVE") + 1
df_new_reviews.insert(insert_index, column="POSITIVE_WORDS", value = positive_words_vect)
df_new_reviews.insert(insert_index, column="NEGATIVE_WORDS", value = negative_words_vect)

# Using pytextrank library to extract key phrases from comments 
$\color{red}{\text{Snowflake Anaconda channel doesn't support pytextrank yet, we may submit a request}}$

In [6]:
import spacy
import pytextrank

def extract_key_phrases(sentiment_str: PandasSeries[str]) -> PandasSeries[str]:
    # load a spaCy model, depending on language, scale, etc.
    nlp = spacy.load("en_core_web_sm")
    # add PyTextRank to the spaCy pipeline
    nlp.add_pipe("textrank")
    
    res = []
    for text in sentiment_str:
        doc = nlp(text)
        # examine the top-ranked phrases in the document
        key_phrases = [phrase.text for phrase in doc._.phrases]
        res.append(', '.join(key_phrases))    
    return pandas.Series(res)

In [7]:
# Extract key phrases from new reviews
key_phrases_vect = extract_key_phrases(df_new_reviews["PROCESSED_TEXT"])
# Add key phrases to the dataframe
insert_index = df_new_reviews.columns.get_loc("POSITIVE_WORDS") + 1
df_new_reviews.insert(insert_index, column="KEY_PHRASES", value = key_phrases_vect)

# Spacy in-build keyphrase extraction

In [19]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")

def extract_key_phrases(sentiment_str: PandasSeries[str]) -> PandasSeries[str]:    
    res = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    for text in sentiment_str:
        words = []
        doc = nlp(text)
        for token in doc:
            if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
                continue
            if(token.pos_ in pos_tag):
                words.append(token.text)
        most_common_list = Counter(words).most_common(5)
        res.append(', '.join([item[0] for item in most_common_list]))   
    return pandas.Series(res)

In [20]:
# Extract key phrases from new reviews
key_phrases_vect = extract_key_phrases(df_new_reviews["PROCESSED_TEXT"])
# Add key phrases to the dataframe
insert_index = df_new_reviews.columns.get_loc("POSITIVE_WORDS") + 1
df_new_reviews.insert(insert_index, column="KEY_PHRASES_spacy", value = key_phrases_vect)

In [22]:
df_new_reviews.head(100)

,ID,TYPE,SUBJECT,DESCRIPTION,PRIORITY,COMMENT,PROCESSED_TEXT,NEGATIVE,POSITIVE,NEGATIVE_WORDS,POSITIVE_WORDS,KEY_PHRASES_spacy,KEY_PHRASES,ENTITIES,REASON,SCORE,SENTIMENT_RATING,CREATED_AT,UPDATED_AT
0,3280,question,Re: French 1 program,"Hi Cindi,\n\nLet me copy Mary on email and also support to get with you ASAP.\n\nTerry\n\nSent from my iPhone\n\nOn Feb 7, 2018, at 1:43 PM, Cindi Bordelon <cbordelon@nederlandisd.org<mailto:cbordelon@nederlandisd.org>> wrote:\n\nHello Terry - we are having problems with the speaking portion of our French 1 Power Speak course. Could you direct me on who we need to contact for help. The program won't let the kids record themselves in any of the lessons all of a sudden.\nThank you,\nCindi\n\...",normal,"The tech support was wonderful. We were able to fix our problem which was actually occurring on our end but without the help of your tech dept, we wouldn't have figured it out.\r\nThanks so much!",the tech support be wonderful we be able to fix our problem which be actually occur on our end but without the help of your tech dept we would not have figure it out thank so much,0.000146,0.999854,,"much, wonderful, able","tech, support, wonderful, able, problem","your tech dept, the help, our end, our problem, the tech support, it, we, which",,No reason provided,good,1,2018-02-07 19:56:26,2018-02-14 20:01:14
1,7279,question,Unable to load link,This issue was reported from LMS\Educator.\r\nDistrict ID = 10233\r\nSchool ID = 35973\r\nSchool Name = TURNER HIGH SCHOOL\r\nTeacher User ID = 325098757\r\nPhone = \r\nSession ID = 17460294\r\nStudent Name = Antonio Hernandez\r\nStudent User ID = 325140455\r\nCourse Name = TUSD Note Taking \r\nCourse ID = 13e3f7fd-c8a3-4221-abbe-5b2eae929d4c\r\nCourse Node ID = 3cbee072-8b26-4275-a7c2-7fe9a3d7a6f5\r\nActivity = TUSD Note Taking > Taking High Quality Notes > Compare and Contrast Graphic Org...,normal,"Very Professional. Try to get back to me for day or two, which is cool. Thank you",very professional try to get back to i for day or two which be cool thank you,0.000281,0.999719,,"very, professional, cool","professional, try, day, cool","day, very professional try, i, two, which, you","day | DATE, two | CARDINAL",No reason provided,good,1,2018-02-23 17:52:06,2018-03-24 17:02:03
2,9458,question,Cant move forward,This issue was reported from LMS\Educator.\r\nDistrict ID = 7359\r\nSchool ID = 23280\r\nSchool Name = TAMPA BAY TECHNICAL HIGH SCHOOL\r\nTeacher User ID = 8883358\r\nPhone = 813-545-1678\r\nSession ID = 17493555\r\nStudent Name = JADA CAMPBELL\r\nStudent User ID = 4755237\r\nCourse Name = NEW 2017-18 FL-1207300-Liberal Arts Mathematics 1 Sem 1 Credit Recovery\r\nCourse ID = 2afe0174-e5c9-4551-a3e4-5cc1cee37b46\r\nCourse Node ID = 1278521d-bdeb-e511-80c3-ecf4bbc39c64\r\nActivity = NEW 2017-1...,normal,The response time was good and the tip is helpful for current and future students.,the response time be good and the tip be helpful for current and future student,0.003221,0.996779,,good,"response, time, good, tip, helpful","current and future student, the response time, the tip",,No reason provided,good,1,2018-03-05 17:35:52,2018-03-09 19:01:04
3,9968,question,Student's having to restart their assignments from the beginning,"Call from: +1 (844) 632-5682\nTime of call: March 06, 2018 02:24:55 PM\nAnswered by: Wendy Reyes",normal,Wendy was very helpful and solved our issue that the teacher/students have been dealing with for the past 14 weeks. Thank you very much!,wendy be very helpful and solve our issue that the teacher student have be deal with for the past 14 week thank you very much,0.010145,0.989855,past,"very, much","wendy, helpful, issue, teacher, student","the teacher student, the past 14 week, wendy, our issue, you","wendy | PERSON, the past 14 week | DATE",No reason provided,good,1,2018-03-06 21:25:42,2018-03-20 19:01:15
4,14361,question,Unable to progress,This issue was reported from LMS\Educator.\r\nDistrict ID = 178\r\nSchool ID = 19244\r\nSchool 

# Entity extraction

In [9]:
import spacy

def extract_entities(sentiment_str: PandasSeries[str]) -> PandasSeries[str]:
    # load a spaCy model, depending on language, scale, etc.
    nlp = spacy.load("en_core_web_sm")
    
    res = []
    for text in sentiment_str:
        doc = nlp(text)
        # examine the entities in the document
        entities = [entity.text + ' | ' + entity.label_ for entity in doc.ents]
        res.append(', '.join(entities))    
    return pandas.Series(res)

In [10]:
# Extract entities from new reviews
entities_vect = extract_entities(df_new_reviews["PROCESSED_TEXT"])
# Add entities to the dataframe
insert_index = df_new_reviews.columns.get_loc("KEY_PHRASES") + 1
df_new_reviews.insert(insert_index, column="ENTITIES", value = entities_vect)

### Write the sentiment result to a new table

In [11]:
scored_reviews_output_path = r"C:\Users\an.jiang\src\snowpark_demo\data\scored_reviews.csv"
df_new_reviews.to_csv(scored_reviews_output_path, index=False)

***

======================================================================================================================

# SQL queries to all the codes above to an automatic pipeline

In [ ]:
# Create a stream on directory table
create stream unstructured_files_stream on stage unstructured_files;

In [ ]:
# Create a stream on the raw_reviews staging table
create stream raw_reviews_stg_stream on table raw_reviews_stg;

In [ ]:
# Create a task to run scoring on the raw reviews

In [ ]:
# Create a task to process the new unstructured files that come in.
create or replace task read_unstructured_reviews_task warehouse = 'wh_xs' schedule = '1 minute' 
when system$stream_has_data('unstructured_files_stream')
as 
insert into raw_reviews_stg
select 
    u.file_url,
    t.product_id,
    t.product_review, 
    t.reviews_date
from 
    unstructured_files_stream u
join 
    table(read_unstructured_reviews(u.file_url)) t
where
    metadata$action = 'INSERT'
;

In [ ]:
# Create a task to process the newly ingested unstructured reviews
create or replace task score_reviews_task warehouse = 'wh_xs' schedule = '1 minute'
when system$stream_has_data('raw_reviews_stg_stream')
as 
insert into annotated.scored_reviews
select
    product_id,
    product_review,
    review_date,
    annotated.predict_sentiment_vect(
        remove_stopword_vect(product_review)) as sentiment
from 
    raw_reviews_stg_stream
where
    metadata$action = 'INSERT'
;

In [ ]:
# Start the tasks
alter task read_unstructured_reviews_task resume;
alter task score_reviews_task resume;